In [33]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 60000
learning_rate = 3e-4
eval_iters=250

cuda


In [3]:
with open('wizard-of-oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')

In [18]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):

        for _ in range(max_new_tokens):

            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples =1)
            index = torch.cat((index, index_next), dim=1)

        #print(index)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device = device)
generated_chars = decode(m.generate(context, 500)[0].tolist())
print(generated_chars)


SA8L)D48bCQyZzP7oU*felD4eEc7Qgp
??,m 4dA2CIk﻿Xv cAnt,GMEGi:VB7A6VxP.Kj&)-M0nI0Y]?mnMdSPBlEaU_1?(tv.vcl(pbyq.paa6Bw]U*NMKK0pwa1ijE5UWHl'7G﻿H4OWjDZfvZrBfQ)AK7xJ6"j9xTza-XeuRueu'Z']VS8L-kq"R.GR&wIQ:owEO4'h?.7[qFdcrD4'ufk4osE04BxX6"go8D4kT49kZD4-i799
m&,Y!OZ]qR)Lr'4OMAlE N&83k﻿[SL-1Z[z]sa-l".:tyHj9-9-bgNvNkT.RuPPPD4kxE MKK5NNManMI,hB5?dz2[1tmAtjZG*!YU_v)OxdLXF*6W2Y8HshihSmWVdO3:N.H5fcAi5-](HQbYLdzCfeuRniG-3MEOiDX]49UWRhzyP9HJqo;R*'O49NL,qm.*tmaVBApGQL.qiazkMzj﻿VjS;3Z-A ﻿wW2D4j2BO]DWR*cr'5CmAYCl)_E&D


In [35]:
@torch.no_grad()
def estimate_loss():
    out ={}
    model.eval()
    for tt in ['train', 'test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(tt)
            logits, loss=model(X,Y)
            losses[k] = loss.item()
        out[tt] = losses.mean()
    model.train()
    return out


In [37]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter%eval_iters==0:
        losses=estimate_loss()
        print(f'step:{iter}, {losses}')
    x, y = get_batch('train')
    logits, loss = model.forward(x,y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss)
print(loss.item())

step:0, {'train': tensor(2.4463), 'test': tensor(2.4688)}
step:250, {'train': tensor(2.4237), 'test': tensor(2.4748)}
step:500, {'train': tensor(2.4194), 'test': tensor(2.4849)}
step:750, {'train': tensor(2.4173), 'test': tensor(2.4669)}
step:1000, {'train': tensor(2.4249), 'test': tensor(2.4591)}
step:1250, {'train': tensor(2.4270), 'test': tensor(2.4418)}
step:1500, {'train': tensor(2.4413), 'test': tensor(2.4747)}
step:1750, {'train': tensor(2.4335), 'test': tensor(2.4745)}
step:2000, {'train': tensor(2.4259), 'test': tensor(2.4811)}
step:2250, {'train': tensor(2.4242), 'test': tensor(2.4935)}
step:2500, {'train': tensor(2.4268), 'test': tensor(2.4659)}
step:2750, {'train': tensor(2.4238), 'test': tensor(2.4802)}
step:3000, {'train': tensor(2.4192), 'test': tensor(2.4803)}
step:3250, {'train': tensor(2.4297), 'test': tensor(2.4793)}
step:3500, {'train': tensor(2.4382), 'test': tensor(2.4537)}
step:3750, {'train': tensor(2.4261), 'test': tensor(2.4818)}
step:4000, {'train': tensor(2.

In [38]:
context = torch.zeros((1,1), dtype=torch.long, device = device)
generated_chars = decode(m.generate(context, 500)[0].tolist())
print(generated_chars)


The wheeanathe arathimuinouthes thuco rd By inousthupthiro thefowasakseyouin's aimeaked tid wond "Bug wenerisileatus am tetaret wed tooug lloin sthe wind ous d."A a r-f
I ca icor, ty. te wrsknd useie fe an wioutou thit igheen  theboorutcly bred ated
Zeedes thinked Mrves st  m ug the  an sng. er andrdind
ghede Thisnt geililk wof r cag f waincound t, m ake eve are on ffrsarakewa ter foul, t, thy
" lendratherfopy


Lad t temy rcenclles ley, tlid awhe, thanoure walt.
ahet orof sllly ore t
t p stouth
